# use another position
- initial_state2, 2nd last is flipped
- has 2 circuits instead of 3 circuits.  11/16/2022

In [2]:
# !pip install pennylane
# !pip install pennylane-qiskit

In [3]:
# !pip install pennylane-lightning

In [4]:
# !pip install pennylane-lightning[gpu]  # has erro

In [5]:
import numpy as np
from pennylane.optimize import AdamOptimizer
from pennylane import numpy as qml_np
import pennylane as qml

import math
import matplotlib.pyplot as plt
import datetime

In [8]:
device_name = 'default.qubit'  #'default.qubit' # 
device_name2 = 'default.qubit' # has qml.state()

In [11]:
coef2 = 1
def objective_fn(theta):
    val1_1 = quantum_circuit_with_HT(theta)
    val1_2 = quantum_circuit_with_HTZ(theta)
    return -0.5*(val1_1 - val1_2)

# 3rd last flipped

In [14]:

N = 2**(num_of_qubits-2)


initial_state3 = [1/math.sqrt(N)]*(N-3) + [0, 1/math.sqrt(N), 1/math.sqrt(N)] + [0]*(N-3) + [1/math.sqrt(N), 0, 0] # 2**(num_qubits-1)
initial_state_phi1 = initial_state3
print(f'initial_state_phi1={initial_state_phi1}')
initial_state_0_phi1  = initial_state_phi1 + [0]*len(initial_state_phi1) # 2**num_of_qubits




initial_state_phi1=[0.5, 0, 0.5, 0.5, 0, 0.5, 0, 0]


In [15]:
## three quantum circuits
def layer_with_HT(theta):
    for i in range(num_of_qubits-1):
        qml.CRY(theta[i], wires=(0,i+1))
    for i in range(num_of_qubits-2):
        qml.Toffoli(wires=(0,i+1,i+2))

        
def layer_no_HT(theta, qubit_posi):
    for i in range(num_of_qubits-1):
        qml.RY(theta[i], wires=(qubit_posi[i]))
    for i in range(num_of_qubits-2):
        qml.CNOT(wires=(qubit_posi[i],qubit_posi[i+1]))
    


In [16]:
device_name = 'default.qubit'  #'default.qubit' # 
device_name2 = 'default.qubit' # has qml.state()

In [17]:

        
# dev_with_HT=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HT=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HT)
def quantum_circuit_with_HT(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
print(qml.draw(quantum_circuit_with_HT)([[0.1]*(num_of_qubits-1)]))





0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│─────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X────┤     


In [18]:

# dev_with_HTZ=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits, shots=20000, backend='qasm_simulator')
dev_with_HTZ=qml.device(device_name, wires=num_of_qubits)
@qml.qnode(dev_with_HTZ)
def quantum_circuit_with_HTZ(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_0_phi1), wires=range(num_of_qubits))
#     qubit_position = list(range(1,num_of_qubits))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits)
    qml.Hadamard(0)
    for theta_i in theta:
        layer_with_HT(theta_i)
    qml.CZ([0,1])
    qml.Hadamard(0)    
    return qml.expval(qml.PauliZ(0)) 
    # return qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
print('newly added')
print(qml.draw(quantum_circuit_with_HTZ)([[0.1]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HTZ([[0.1]*(num_of_qubits-1)]))



# dev_no_HT_Z=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, shots=20000, backend='qasm_simulator')
dev_no_HT_Z=qml.device(device_name, wires=num_of_qubits-1)        
@qml.qnode(dev_no_HT_Z)
def quantum_circuit_no_HT_return_Z(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))
    
    return qml.expval(qml.PauliZ(0))  
    # return qml.sample(qml.PauliZ(0)) 
print('newly added 2')   
print(qml.draw(quantum_circuit_no_HT_return_Z)([[0.2]*(num_of_qubits-1)]))
# print(quantum_circuit_with_HT([[0.2]*(num_of_qubits-1)]))


# dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits+1) #AerDevice(wires=num_of_qubits-1, backend='qasm_simulator')
dev_no_HT_S=qml.device(device_name2, wires=num_of_qubits-1)  
@qml.qnode(dev_no_HT_S)
def quantum_circuit_no_HT_return_state(theta):
    # initiate state vector |phi_1>
    qml.QubitStateVector(np.array(initial_state_phi1), wires=range(num_of_qubits-1))
#     qubit_position = list(range(num_of_qubits-1))
#     initiate_state_0_phi1(qml, qubit_position, work_wires=num_of_qubits-1)
    for theta_i in theta:
        layer_no_HT(theta_i, list(range(num_of_qubits-1)))    
    return qml.state()

print('newly added 3')

newly added
0: ─╭QubitStateVector(M0)──H─╭●────────╭●────────╭●────────╭●─╭●─╭●──H─┤  <Z>
1: ─├QubitStateVector(M0)────╰RY(0.10)─│─────────│─────────├●─│──╰Z────┤     
2: ─├QubitStateVector(M0)──────────────╰RY(0.10)─│─────────╰X─├●───────┤     
3: ─╰QubitStateVector(M0)────────────────────────╰RY(0.10)────╰X───────┤     
newly added 2
0: ─╭QubitStateVector(M0)──RY(0.20)─╭●────┤  <Z>
1: ─├QubitStateVector(M0)──RY(0.20)─╰X─╭●─┤     
2: ─╰QubitStateVector(M0)──RY(0.20)────╰X─┤     
newly added 3


In [20]:
max_repeat = 1 #100
iter_max = 300  #300
num_of_layers = 3
prb_last_list = []
obj_list_rep = []
theta_list = []
debug_flag, print_flag = 0, True
start_time = datetime.datetime.now()
for rep in range(1,max_repeat+1):
    if print_flag:
        print(f'\n\nrep={rep}')
    else:
        print(f'\n\nrep={rep}', end='  ')
    
    optimizer = AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
    theta=qml_np.random.uniform(0, 2*math.pi, size=(num_of_layers, num_of_qubits-1), requires_grad=True)
    obj_list = []
    for iter in range(1, iter_max+1):    
        theta, obj = optimizer.step_and_cost(objective_fn, theta)
        val1_1 = quantum_circuit_with_HT(theta)
        val1_2 = quantum_circuit_with_HTZ(theta)
        val2 = quantum_circuit_no_HT_return_Z(theta)
        if (iter==1 or iter%20==0 or iter==iter_max) and print_flag:
            print(f'iter={iter:3d} :: obj={obj:12.8f} :: val1_1={val1_1:12.8f} \
                 :: val1_2={val1_2:12.8f} :: val2={val2:12.8f}  \
                 coef2*val2-0.5*(val1_1 - val1_2)={coef2*val2-0.5*(val1_1 - val1_2):12.8f}  \
                 :: theta={theta}')
            # print(f'iter={iter:3d} :: obj={obj:12.8f} :: theta={theta}')
        obj_list.append(obj)
    theta_list.append(theta)
    print(f'obj_list(last 3)={obj_list[-3:]}')
    obj_list_rep.append(obj_list)
    
    ## display the amplified state
    state = quantum_circuit_no_HT_return_state(theta)
    prb = [i.item()**2 for i in state]
    # print(f'state={state}')
    if len(prb)>20:
        print(f'prb(last 3)={prb[-3:]}')
    else: print(f'prb={prb}')
    prb_last_list.append(prb[-3])
end_time = datetime.datetime.now()
duration = end_time - start_time
duration_in_s = duration.total_seconds()
print(f'time consumed: {duration_in_s}s')



rep=1
iter=  1 :: obj= -0.20109512 :: val1_1=  0.33914298                  :: val1_2= -0.12906753 :: val2=  0.44324409                   coef2*val2-0.5*(val1_1 - val1_2)=  0.20913884                   :: theta=[[3.99739049 5.68831983 3.95643202]
 [3.99046664 1.18555397 5.20029614]
 [2.56423048 3.66516537 5.35397754]]
iter= 20 :: obj= -0.46741621 :: val1_1=  0.52173836                  :: val1_2= -0.41619065 :: val2= -0.79112939                   coef2*val2-0.5*(val1_1 - val1_2)= -1.26009389                   :: theta=[[3.3036794  6.31140155 4.02203203]
 [3.14958868 2.01800679 4.79906547]
 [2.04482671 3.86803583 4.85855529]]
iter= 40 :: obj= -0.49075962 :: val1_1=  0.42674793                  :: val1_2= -0.55718504 :: val2= -0.94350200                   coef2*val2-0.5*(val1_1 - val1_2)= -1.43546849                   :: theta=[[3.51044698 5.78114399 3.59980329]
 [2.88383334 2.3271018  5.20533672]
 [2.37870753 4.06935148 4.7702508 ]]
iter= 60 :: obj= -0.49948379 :: val1_1=  0.47947535  

In [21]:
print('time now: ', end_time)

time now:  2022-11-16 09:52:54.879305
